# Final Assignment

Author: Liam Hoogstad  
CID: 01956599  
Date: 13 November 2025

## 1. Importing Libraries

In [4]:
# Loading required libraries
#| warning: false
# install.packages(c("dplyr", "ggplot2", "PerformanceAnalytics"))
# install.packages(c("glue"))
# install.packages(c("lubridate"))
library(dplyr)
library(ggplot2)
library(PerformanceAnalytics)
library(lubridate)
library(glue)

## 2. Data Cleaning

First, I need to select the rows from the csv file containing Pfizer's information.  
I'm going to do this by selecting the rows with tic == 'PFE'

In [ ]:
# Read the CSV file
compustat_data <- read.csv("compustat_daily_2010_2025.csv")

# Filter for PFE ticker
pfe_data <- compustat_data %>%
  filter(tic == "PFE") %>%
  # removing unnecessary columns
  select(-add1, -addzip, -busdesc, -city, -conml, -fax, -loc, -phone, -weburl)

head(pfe_data, n = 10) # first 10 rows

# saving filtered data to a new csv file
write.csv(pfe_data, "pfe_data.csv", row.names = FALSE)

,tic,datadate,conm,exchg,sic,cshtrd,prccd,prchd,prcld,prcod,gvkey
,<chr>,<chr>,<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
1,PFE,01/04/2010,PFIZER INC,11,2834,52074710,18.93,18.94,18.235,18.27,8530
2,PFE,01/05/2010,PFIZER INC,11,2834,43368460,18.66,18.93,18.550,18.92,8530
3,PFE,01/06/2010,PFIZER INC,11,2834,41405070,18.60,18.81,18.510,18.66,8530
4,PFE,01/07/2010,PFIZER INC,11,2834,39427720,18.53,18.67,18.460,18.64,8530
5,PFE,01/08/2010,PFIZER INC,11,2834,30403370,18.68,18.71,18.520,18.62,8530
6,PFE,01/11/2010,PFIZER INC,11,2834,32442710,18.83,18.95,18.670,18.83,8530
7,PFE,01/12/2010,PFIZER INC,11,2834,41270900,18.77,18.99,18.640,18.80,8530
8,PFE,01/13/2010,PFIZER INC,11,2834,59506750,19.21,19.30,18.870,18.87,8530
9,PFE,01/14/2010,PFIZER INC,11,2834,47258330,19.38,19.50,19.130,19.20,8530
